In [ ]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import json
import os
import sys
load_dotenv()


In [26]:
## SSL Certificate - local solution for SSL encryption

#Authentication

token = os.getenv("emplify_token") # this is fetching my username from .env file - to be moved to azure vault from .env on deployment

secret = os.getenv("emplify_secret") # this is fetching my password from .env file - to be moved to azure vault from .env on deployment

cert_path = os.getenv("SSL_cert_loc") # this is fetching an SSL certificate due - is a workaround for local setup and can be removed on deployment

auth = (token, secret)


In [27]:
## Dim_social_profiles

networks = ['facebook', 'instagram', 'twitter', 'youtube', 'tiktok'] # selection of s_media accounts
profiles = pd.DataFrame()  # empty dataframe initiation

for n in networks:
    url = 'https://api.emplifi.io/3/'+n+'/profiles'
    body = ''
    r = requests.get(url, auth = auth, verify = cert_path)
    req_json = json.loads(r.text)
    new_row_df = pd.json_normalize(req_json, 'profiles', ['success'])
    new_row_df['network'] = n
    profiles = pd.concat([profiles, new_row_df], ignore_index = True) #profiles dataframe holds list of social media profiles metrics will be fetched for. To be exported as dim_social_profiles
    
    

In [28]:
#Creating a list of metrics to pull and declaring start and end date
agg_mettrics = ['insights_impressions','insights_video_views','insights_engagements','likes','number_of_comments']
start_date = '2024-07-03'
end_date = '2024-10-08'


In [ ]:
## Filtering to those who have insights enabled and getting a dictionary as an input

profiles_filter = profiles[(profiles['profile_name'].str.lower().str.contains('triumph')) & (profiles['insights_enabled'] == True)]

profiles2 = profiles_filter.rename(columns={'network':'platform'})

prof = profiles2[['id', 'platform']].to_dict('records')
print ("Profiles list fetched successfully" )
prof


In [30]:
## Function to fetch aggregated metrics at profile level

def fetch_insights_data(profiles, date_start, date_end, metric):
    body = {
        "profiles": profiles,
        "date_start": date_start,
        "date_end": date_end,
        "metric": metric,
        "dimensions": [
            {"type": "date.day"},
            {"type": "profile"}
        ]
    }
    url = 'https://api.emplifi.io/3/aggregated-metrics'
    
   
    response = requests.post(url, json=body, auth=auth, verify=cert_path)
    return json.loads(response.text)


In [31]:
# Function to tranform metrics json at profile level
def transform_dict(dict):
    dates = dict['header'][0]['rows']
    profiles = [f"{p['id']}" for p in dict['header'][1]['rows']]
    platform = [f"({p['platform']})" for p in dict['header'][1]['rows']]
    metric = dict['header'][2]['rows'][0]
    data = dict['data']

# Preparing data for DataFrame
    df_data = {
        'profile': [],
        'date': [],
        'metric': [],
        'data': []
    }

    for i, profile in enumerate(profiles):
        for j, date in enumerate(dates):
            df_data['profile'].append(profile)
            df_data['date'].append(date)
            df_data['metric'].append(metric)
            df_data['data'].append(data[j][i][0] if data[j][i] else None)

    # Creating DataFrame
    df = pd.DataFrame(df_data)
    return df

In [ ]:
#Aggregated Profile metrics
df_list = []  # Initialize an empty list


for met in agg_mettrics:
    dat = transform_dict(fetch_insights_data(prof,start_date,end_date,met)) # utilising prof list of profiles, met as metric for each item in agg_metrics
    df_list.append(dat)  # Append the data frame to the list


    
# Concatenate all data frames in the list
prof_lev_met_agg = pd.concat(df_list) 
prof_lev_met_agg

# Post Level Metrics

In [ ]:
##Pulling in Post Labels - for dim_posts table
url = 'https://api.emplifi.io/3/post/labels'
r = requests.get(url, auth = auth, verify = cert_path)
posts = json.loads(r.text)

data = posts['data']        

# Creating DataFrame
posts = pd.DataFrame(data)
posts


In [34]:
## Function fetching post level metrics 

def fetch_insights_data_post_lev(profiles, date_start, date_end, metric):
    body = {
        "profiles": profiles,
        "date_start": date_start,
        "date_end": date_end,
        "metric": metric,
        "dimensions": [
            {"type": "date.day"},
           
            {"type": "post_labels"}
        ]
    }
    url = 'https://api.emplifi.io/3/aggregated-metrics'
    
   
    response = requests.post(url, json=body, auth=auth, verify=cert_path)
    return json.loads(response.text)

In [35]:
# Function transforming data frame with post level metrics

def transform_dict_post(dict):
    dates = dict['header'][0]['rows']
    post_labels = dict['header'][1]['rows']
    metric = dict['header'][2]['rows'][0]
    data = dict['data']

    # Preparing data for DataFrame
    df_data = {
        'date': [],
        'post_label': [],
        'metric': [],
        'data': []
    }

    for i, post_label in enumerate(post_labels):
        for j, date in enumerate(dates):
            df_data['date'].append(date)
            df_data['post_label'].append(post_label)
            df_data['metric'].append(metric)
            df_data['data'].append(data[j][i][0] if data[j][i] else None)

    # Creating DataFrame
    df = pd.DataFrame(df_data)
    return df

In [36]:
# Main Run of function that fetches and transforms dataframe with Post (fact_posts)
df_list = []  # Initialize an empty list

for met in agg_mettrics:
    dat = transform_dict_post(fetch_insights_data_post_lev(prof,start_date,end_date,met))
    df_list.append(dat)  # Append the data frame to the list

    
# Concatenate all data frames in the list
final_df = pd.concat(df_list) 

In [ ]:
final_df